In [1]:
!pip install stable-baselines3[extra]
!pip install gym --upgrade

In [12]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [13]:
environment_name = 'CartPole-v1'
env = gym.make(environment_name, render_mode="human")

In [14]:
environment_name

'CartPole-v1'

In [5]:
episodes = 5
for episode in range (1,episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        step_output = env.step(action)
        n_state = step_output[0]  # Next state
        reward = step_output[1]    # Reward
        done = step_output[2]      # Done flag
        info = step_output[4] # Additional info dictionary
        score += reward
    print('Episode:{} Score:{}'.format(episode,score))
#env.close()

C:\Users\Diogo\anaconda3\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode:1 Score:13.0
Episode:2 Score:21.0
Episode:3 Score:27.0
Episode:4 Score:21.0
Episode:5 Score:24.0


In [15]:
#MAKE YOUR DIRECTORIES FIRST
log_path = os.path.join('Training', 'Logs')

In [16]:
log_path

'Training\\Logs'

In [17]:
env = gym.make(environment_name, render_mode="human")
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


C:\Users\Diogo\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [9]:
PPO??

Init signature:
PPO(
    policy: Union[str, Type[stable_baselines3.common.policies.ActorCriticPolicy]],
    env: Union[gymnasium.core.Env, ForwardRef('VecEnv'), str],
    learning_rate: Union[float, Callable[[float], float]] = 0.0003,
    n_steps: int = 2048,
    batch_size: int = 64,
    n_epochs: int = 10,
    gamma: float = 0.99,
    gae_lambda: float = 0.95,
    clip_range: Union[float, Callable[[float], float]] = 0.2,
    clip_range_vf: Union[NoneType, float, Callable[[float], float]] = None,
    normalize_advantage: bool = True,
    ent_coef: float = 0.0,
    vf_coef: float = 0.5,
    max_grad_norm: float = 0.5,
    use_sde: bool = False,
    sde_sample_freq: int = -1,
    rollout_buffer_class: Optional[Type[stable_baselines3.common.buffers.RolloutBuffer]] = None,
    rollout_buffer_kwargs: Optional[Dict[str, Any]] = None,
    target_kl: Optional[float] = None,
    stats_window_size: int = 100,
    tensorboard_log: Optional[str] = None,
    policy_kwargs: Optional[Dict[str, Any]]

In [18]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_15
-----------------------------
| time/              |      |
|    fps             | 44   |
|    iterations      | 1    |
|    time_elapsed    | 46   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 44          |
|    iterations           | 2           |
|    time_elapsed         | 91          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008610891 |
|    clip_fraction        | 0.107       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00033     |
|    learning_rate        | 0.0003      |
|    loss                 | 7.33        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0171     |
|    value_loss           | 51          |
-----------------------------------------
--

In [23]:
##SAVE AND RELOAD MODEL##
PPO_Path = os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')

In [24]:
model.save(PPO_Path)

In [13]:
del model

In [14]:
model = PPO.load(PPO_Path, env=env)

In [15]:
model.learn(total_timesteps=1000)

Logging to Training\Logs\PPO_7
-----------------------------
| time/              |      |
|    fps             | 716  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------


In [35]:
##EVALUATION##
evaluate_policy(model, env, n_eval_episodes=10, render= True)

(500.0, 0.0)

In [27]:
env.close()

In [1]:
##TEST MODEL###

episodes = 5
for episode in range (1,episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode,score))
#env.close()

NameError: name 'env' is not defined

In [39]:
obs = env.reset()

In [45]:
action, _ = model.predict(obs)

In [43]:
env.action_space.sample()

0

In [44]:
obs

array([[-0.00996539,  0.03280865,  0.03334983, -0.00674351]],
      dtype=float32)

In [46]:
env.step(action)

(array([[-0.00930922, -0.16277532,  0.03321496,  0.29627234]],
       dtype=float32),
 array([1.], dtype=float32),
 array([False]),
 [{'TimeLimit.truncated': False}])

In [48]:
##TENSORBOARD##
training_log_path = os.path.join(log_path, 'PPO_9')

In [49]:
training_log_path

'Training\\Logs\\PPO_9'

In [50]:
!tensorboard --logdir={training_log_path}

^C


In [54]:
##ADDING CALLBACK TO THE TRAINING STAGE##

from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [56]:
save_path = os.path.join('Training', 'Saved Models')

In [57]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=500, verbose=1)
eval_callback = EvalCallback(env, callback_on_new_best=stop_callback, eval_freq=10000, best_model_save_path=save_path, verbose=1)

In [58]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [59]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_10
-----------------------------
| time/              |      |
|    fps             | 46   |
|    iterations      | 1    |
|    time_elapsed    | 44   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 45          |
|    iterations           | 2           |
|    time_elapsed         | 90          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008868981 |
|    clip_fraction        | 0.0876      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.000701   |
|    learning_rate        | 0.0003      |
|    loss                 | 5.86        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0147     |
|    value_loss           | 52.8        |
-----------------------------------------
--

C:\Users\Diogo\anaconda3\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=348.80 +/- 141.20
Episode length: 348.80 +/- 141.20
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 349         |
|    mean_reward          | 349         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.007414636 |
|    clip_fraction        | 0.0742      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.611      |
|    explained_variance   | 0.297       |
|    learning_rate        | 0.0003      |
|    loss                 | 28.8        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0164     |
|    value_loss           | 61.1        |
-----------------------------------------
New best mean reward!
------------------------------
| time/              |       |
|    fps             | 38    |
|    iterations      | 5     |
|    

In [62]:
##CHANGING POLICIES##
net_arch = [dict(pi=[128,128,128,128], vf=[128,128,128,128])]

In [63]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch': net_arch})

Using cpu device


C:\Users\Diogo\anaconda3\Lib\site-packages\stable_baselines3\common\policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [64]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_11
-----------------------------
| time/              |      |
|    fps             | 46   |
|    iterations      | 1    |
|    time_elapsed    | 43   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 44          |
|    iterations           | 2           |
|    time_elapsed         | 91          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.013663128 |
|    clip_fraction        | 0.207       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.682      |
|    explained_variance   | -0.00124    |
|    learning_rate        | 0.0003      |
|    loss                 | 3.42        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.024      |
|    value_loss           | 20.6        |
-----------------------------------------
--

C:\Users\Diogo\anaconda3\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=457.60 +/- 79.41
Episode length: 457.60 +/- 79.41
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 458         |
|    mean_reward          | 458         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.010379459 |
|    clip_fraction        | 0.0869      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.557      |
|    explained_variance   | 0.433       |
|    learning_rate        | 0.0003      |
|    loss                 | 11.9        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.016      |
|    value_loss           | 44.5        |
-----------------------------------------
------------------------------
| time/              |       |
|    fps             | 36    |
|    iterations      | 5     |
|    time_elapsed    | 280   

In [65]:
##USING AN ALTERNATE ALGORITHM##

from stable_baselines3 import DQN

In [68]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [69]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.959    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 46       |
|    time_elapsed     | 1        |
|    total_timesteps  | 87       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.925    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 45       |
|    time_elapsed     | 3        |
|    total_timesteps  | 158      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.512    |
|    n_updates        | 14       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.878    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 

In [19]:
env.close()